<a href="https://colab.research.google.com/github/AUMANSH/Data-Science-Projects/blob/main/Write_the_ANN_for_house_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/Housing.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [ ]:
# Identify categorical columns
categorical_cols = X_train.select_dtypes(include='object').columns
print("Categorical columns:", categorical_cols)

# Apply one-hot encoding
X_train_encoded = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# Ensure columns are consistent after one-hot encoding
# This is important because the test set might not have all categories present in the training set
train_cols = X_train_encoded.columns
test_cols = X_test_encoded.columns

missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    X_test_encoded[c] = 0
# Ensure the order of columns is the same
X_test_encoded = X_test_encoded[train_cols]


Categorical columns: Index(['mainroad', 'guestroom', 'basement', 'hotwaterheating',
       'airconditioning', 'prefarea', 'furnishingstatus'],
      dtype='object')


In [ ]:
from sklearn.preprocessing import StandardScaler
# Apply StandardScaler to the encoded data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

print("\nShape of scaled training data:", X_train_scaled.shape)
print("Shape of scaled test data:", X_test_scaled.shape)



Shape of scaled training data: (436, 13)
Shape of scaled test data: (109, 13)


In [ ]:
X_train_scaled

array([[ 0.38416819,  0.05527092,  1.53917323, ..., -0.55262032,
        -0.870669  , -0.67690027],
       [ 0.9291807 ,  0.05527092,  1.53917323, ..., -0.55262032,
         1.14854209, -0.67690027],
       [-0.60775457, -1.28351359, -0.5579503 , ..., -0.55262032,
        -0.870669  , -0.67690027],
       ...,
       [-0.29709744,  0.05527092,  1.53917323, ..., -0.55262032,
        -0.870669  , -0.67690027],
       [-0.5060189 , -1.28351359, -0.5579503 , ..., -0.55262032,
        -0.870669  ,  1.47732249],
       [ 0.15707965,  0.05527092,  1.53917323, ..., -0.55262032,
         1.14854209, -0.67690027]])

In [ ]:
X_test_scaled

array([[ 0.33875048,  1.39405543,  1.53917323, ..., -0.55262032,
        -0.870669  ,  1.47732249],
       [ 0.61125674,  0.05527092,  1.53917323, ...,  1.80956067,
        -0.870669  , -0.67690027],
       [-0.5060189 , -1.28351359, -0.5579503 , ..., -0.55262032,
         1.14854209, -0.67690027],
       ...,
       [ 0.38416819,  1.39405543,  1.53917323, ..., -0.55262032,
         1.14854209, -0.67690027],
       [ 0.38416819,  0.05527092,  1.53917323, ..., -0.55262032,
         1.14854209, -0.67690027],
       [ 0.4295859 ,  0.05527092,  1.53917323, ...,  1.80956067,
        -0.870669  , -0.67690027]])

In [ ]:
y_test

,price
316,4060000
77,6650000
360,3710000
90,6440000
493,2800000
...,...
15,9100000
357,3773000
39,7910000
54,7350000


In [ ]:
y_test

,price
316,4060000
77,6650000
360,3710000
90,6440000
493,2800000
...,...
15,9100000
357,3773000
39,7910000
54,7350000


In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))  # linear for regression

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,161 (47.50 KB)

 Trainable params: 12,161 (47.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
history = model.fit(X_train_scaled,y_train,batch_size=64,epochs=300,verbose=1,validation_split=0.2) # verbose = 0 to code ki working nahi dikhegi

Epoch 1/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 25181703634944.0000 - mae: 4713085.0000 - val_loss: 24781594296320.0000 - val_mae: 4623896.5000
Epoch 2/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 26044532785152.0000 - mae: 4792648.5000 - val_loss: 24781583810560.0000 - val_mae: 4623896.0000
Epoch 3/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 24827693891584.0000 - mae: 4692116.5000 - val_loss: 24781573324800.0000 - val_mae: 4623894.5000
Epoch 4/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 25221406916608.0000 - mae: 4719422.5000 - val_loss: 24781556547584.0000 - val_mae: 4623892.5000
Epoch 5/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 26145774895104.0000 - mae: 4802164.5000 - val_loss: 24781529284608.0000 - val_mae: 4623890.0000
Epoch 6/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 26001702649856.0000 - mae: 4781440.5000 - val_loss: 24781493633024.0000 - val_mae: 4623886.5000
Epoch 7/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 25352208384000.000

In [ ]:
y_pred = model.predict(X_test_scaled)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [ ]:
y_pred

array([[ 1719452.1],
       [ 7543292. ],
       [ 3825104.8],
       [ 4011628.2],
       [ 3061239. ],
       [ 3208527.8],
       [ 4799444. ],
       [ 6293549. ],
       [ 1959690.2],
       [ 2678201.2],
       [13421124. ],
       [ 3891025.8],
       [ 2510154. ],
       [ 2072732.9],
       [ 3037197. ],
       [ 1085987.4],
       [ 3603428.2],
       [ 3265422. ],
       [ 2987878. ],
       [ 3809761.8],
       [ 5111127.5],
       [ 6112207. ],
       [ 3911659. ],
       [ 3835986.5],
       [ 4594052.5],
       [ 8209779. ],
       [ 3018020.2],
       [ 3921261.8],
       [ 9553347. ],
       [ 3299519.8],
       [ 6605546.5],
       [ 2706256.5],
       [ 6392509. ],
       [ 3667473.5],
       [ 2821146.5],
       [ 5535641.5],
       [ 3874798.2],
       [ 3646294.5],
       [ 2038203. ],
       [ 3625241.8],
       [ 3753459. ],
       [ 3214127. ],
       [ 7514263. ],
       [ 2765089.5],
       [ 3834585.2],
       [ 2842994.8],
       [ 7306503.5],
       [ 3613

In [ ]:
y_pred = (y_pred > 0.5).astype(int)

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)


MAE: 5007535.697247706


In [ ]:
import pickle
pickle.dump(model,open('model.pkl','wb'))